# Bayesian Competing Risks Model

This notebook implements the **Bayesian competing risks proportional hazards model** from Bhattacharya, Wilson & Soyer (2019) for mortgage default and prepayment prediction.

## Methodology

| Aspect | Description |
|--------|-------------|
| **Model** | Bayesian competing risks PHM with lognormal baseline hazards |
| **Inference** | MCMC via Pyro/NUTS (PyTorch-based) |
| **Features** | Loan characteristics (matching Blumenstock et al. 2022) |
| **Evaluation** | Time-dependent C-index at 24, 48, 72 months |

## Key Advantages of Bayesian Approach

| Feature | Description |
|---------|-------------|
| **Full uncertainty quantification** | Posterior distributions for all parameters |
| **Probabilistic predictions** | Credible intervals for survival/CIF |
| **Prior information** | Can incorporate domain knowledge |
| **Coherent inference** | Natural handling of small samples |

## Model Specification

**Default hazard:**
$$\lambda_D(t | X) = r_D(t | \mu_D, \sigma_D) \exp(\theta_D' X)$$

**Prepayment hazard:**
$$\lambda_P(t | X) = r_P(t | \mu_P, \sigma_P) \exp(\theta_P' X)$$

where $r(t | \mu, \sigma)$ is the lognormal baseline hazard.

## References

- Bhattacharya, A., Wilson, S.P., & Soyer, R. (2019). A Bayesian approach to modeling mortgage default and prepayment. *European Journal of Operational Research*, 274, 1112-1124.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Bayesian inference (Pyro - PyTorch based)
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az

# Survival analysis metrics
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored, concordance_index_ipcw

# Preprocessing
from sklearn.preprocessing import StandardScaler
import pickle

# Import our Bayesian evaluation module
import sys
sys.path.insert(0, '../src')
from competing_risks.bayesian_evaluation import (
    compute_time_dependent_cindex,
    compute_brier_score,
    compute_calibration,
    compute_coverage_probability,
    evaluate_bayesian_model,
    format_evaluation_results,
)

sns.set_style('whitegrid')
%matplotlib inline

# Set random seeds
np.random.seed(42)
pyro.set_rng_seed(42)

# Time horizons for evaluation (matching previous notebooks)
TIME_HORIZONS = [24, 48, 72]

# Device selection
if torch.cuda.is_available():
    DEVICE = 'cuda'
elif torch.backends.mps.is_available():
    DEVICE = 'mps'
else:
    DEVICE = 'cpu'

print(f"PyTorch version: {torch.__version__}")
print(f"Pyro version: {pyro.__version__}")
print(f"Device: {DEVICE}")
print(f"Time horizons for C-index evaluation: {TIME_HORIZONS} months")

In [ ]:
# === CONFIGURATION ===
DATA_DIR = Path('../data/processed')
FIGURES_DIR = Path('../reports/figures')
MODELS_DIR = Path('../models')

FIGURES_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Cross-validation folds (Blumenstock methodology)
TRAIN_FOLDS = list(range(9))   # Folds 0-8 for training
VAL_FOLDS = [9]                 # Fold 9 for validation
TEST_FOLD = 10                  # Fold 10 for testing

# MCMC settings
MCMC_PARAMS = {
    'num_warmup': 1000,
    'num_samples': 2000,
    'num_chains': 4,
    'target_accept_prob': 0.8,
}

# Prior hyperparameters (from Bhattacharya et al. 2018)
PRIOR_PARAMS = {
    'theta_sd': 100.0,      # SD for regression coefficient priors
    'mu_sd': 10.0,          # SD for baseline mean priors
    'sigma_rate': 0.01,     # Rate for exponential prior on sigma (mean=100)
}

print(f"Training folds: {TRAIN_FOLDS}")
print(f"Validation fold: {VAL_FOLDS}")
print(f"Test fold: {TEST_FOLD}")
print(f"\nMCMC parameters:")
for k, v in MCMC_PARAMS.items():
    print(f"  {k}: {v}")
print(f"\nPrior parameters:")
for k, v in PRIOR_PARAMS.items():
    print(f"  {k}: {v}")

---

## Load and Prepare Data

In [ ]:
# Load the loan-month panel data
print("Loading loan-month panel data...")
panel_df = pd.read_parquet(DATA_DIR / 'loan_month_panel.parquet')

print(f"Loaded {len(panel_df):,} loan-months")
print(f"Unique loans: {panel_df['loan_sequence_number'].nunique():,}")
print(f"Folds: {sorted(panel_df['fold'].unique())}")
print(f"Vintages: {panel_df['vintage_year'].min()} - {panel_df['vintage_year'].max()}")

print("\nEvent distribution (terminal observations):")
event_names = {0: 'Censored', 1: 'Prepay', 2: 'Default'}
terminal_events = panel_df[panel_df['event'] == 1].groupby('event_code').size()
for code, count in terminal_events.items():
    print(f"  {event_names.get(code, 'Other')} (k={code}): {count:,}")

In [ ]:
# Define feature groups
STATIC_FEATURES = ['int_rate', 'orig_upb', 'fico_score', 'dti_r', 'ltv_r']
BEHAVIORAL_FEATURES = ['bal_repaid', 't_act_12m', 't_del_30d_12m', 't_del_60d_12m']
MACRO_FEATURES = ['hpi_st_d_t_o', 'ppi_c_FRMA', 'TB10Y_d_t_o', 'FRMA30Y_d_t_o']

ALL_FEATURES = STATIC_FEATURES + BEHAVIORAL_FEATURES + MACRO_FEATURES
feature_cols = [f for f in ALL_FEATURES if f in panel_df.columns]
print(f"Available features: {len(feature_cols)}/{len(ALL_FEATURES)}")

In [ ]:
# Prepare terminal observations
time_col, event_col = 'loan_age', 'event_code'
panel_df = panel_df.sort_values(['loan_sequence_number', time_col])
terminal_df = panel_df.groupby('loan_sequence_number').last().reset_index()

# Feature engineering
if 'bal_repaid' in feature_cols:
    bal_repaid_lag = panel_df.groupby('loan_sequence_number').apply(
        lambda g: g['bal_repaid'].iloc[-2] if len(g) >= 2 else g['bal_repaid'].iloc[-1])
    terminal_df['bal_repaid_lag1'] = terminal_df['loan_sequence_number'].map(bal_repaid_lag)
    feature_cols = [f if f != 'bal_repaid' else 'bal_repaid_lag1' for f in feature_cols]

if 'orig_upb' in terminal_df.columns:
    terminal_df['log_upb'] = np.log(terminal_df['orig_upb'])
    feature_cols = [f if f != 'orig_upb' else 'log_upb' for f in feature_cols]

terminal_df = terminal_df.dropna(subset=feature_cols)
print(f"Terminal observations: {len(terminal_df):,}")

In [ ]:
# Split data
train_df = terminal_df[terminal_df['fold'].isin(TRAIN_FOLDS)].copy()
val_df = terminal_df[terminal_df['fold'].isin(VAL_FOLDS)].copy()
test_df = terminal_df[terminal_df['fold'] == TEST_FOLD].copy()

print(f"Train: {len(train_df):,}, Val: {len(val_df):,}, Test: {len(test_df):,}")

In [ ]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[feature_cols]).astype('float32')
X_val = scaler.transform(val_df[feature_cols]).astype('float32')
X_test = scaler.transform(test_df[feature_cols]).astype('float32')

duration_train = np.maximum(train_df[time_col].values.astype('float32'), 0.5)
duration_val = np.maximum(val_df[time_col].values.astype('float32'), 0.5)
duration_test = np.maximum(test_df[time_col].values.astype('float32'), 0.5)

event_train = train_df[event_col].values.astype('int32')
event_val = val_df[event_col].values.astype('int32')
event_test = test_df[event_col].values.astype('int32')

print(f"X_train: {X_train.shape}, Duration range: {duration_train.min():.1f}-{duration_train.max():.1f}")

---

## Bayesian Competing Risks Model

In [ ]:
# Lognormal hazard functions
def lognormal_log_hazard(t, mu, sigma):
    z = (torch.log(t) - mu) / sigma
    log_phi = -0.5 * z**2 - 0.5 * torch.log(torch.tensor(2 * np.pi, device=t.device))
    Phi_z = 0.5 * (1 + torch.erf(z / np.sqrt(2)))
    log_survival = torch.log(1 - Phi_z + 1e-10)
    return log_phi - torch.log(sigma) - torch.log(t) - log_survival

def lognormal_cumulative_hazard(t, mu, sigma):
    z = (torch.log(t) - mu) / sigma
    Phi_z = 0.5 * (1 + torch.erf(z / np.sqrt(2)))
    return -torch.log(1 - Phi_z + 1e-10)

print("Hazard functions defined.")

In [ ]:
# Pyro model
def bayesian_competing_risks_model(X, durations, events, n_features):
    device = X.device
    
    # Priors
    mu_D = pyro.sample('mu_D', dist.Normal(3.0, PRIOR_PARAMS['mu_sd']))
    sigma_D = pyro.sample('sigma_D', dist.Exponential(PRIOR_PARAMS['sigma_rate']))
    mu_P = pyro.sample('mu_P', dist.Normal(3.0, PRIOR_PARAMS['mu_sd']))
    sigma_P = pyro.sample('sigma_P', dist.Exponential(PRIOR_PARAMS['sigma_rate']))
    
    theta_D = pyro.sample('theta_D', dist.Normal(
        torch.zeros(n_features, device=device),
        PRIOR_PARAMS['theta_sd'] * torch.ones(n_features, device=device)).to_event(1))
    theta_P = pyro.sample('theta_P', dist.Normal(
        torch.zeros(n_features, device=device),
        PRIOR_PARAMS['theta_sd'] * torch.ones(n_features, device=device)).to_event(1))
    
    # Linear predictors
    eta_D = torch.matmul(X, theta_D)
    eta_P = torch.matmul(X, theta_P)
    
    # Hazards
    log_h_D = lognormal_log_hazard(durations, mu_D, sigma_D) + eta_D
    log_h_P = lognormal_log_hazard(durations, mu_P, sigma_P) + eta_P
    H_D = lognormal_cumulative_hazard(durations, mu_D, sigma_D) * torch.exp(eta_D)
    H_P = lognormal_cumulative_hazard(durations, mu_P, sigma_P) * torch.exp(eta_P)
    
    # Log-likelihood
    log_lik = (events == 2).float() * log_h_D + (events == 1).float() * log_h_P - H_D - H_P
    pyro.factor('log_likelihood', torch.sum(log_lik))

print(f"Model parameters: {4 + 2*len(feature_cols)}")

---

## MCMC Inference

In [ ]:
# Run MCMC
print(f"Running MCMC on {DEVICE}...")
pyro.clear_param_store()

X_train_torch = torch.tensor(X_train, dtype=torch.float32, device=DEVICE)
duration_train_torch = torch.tensor(duration_train, dtype=torch.float32, device=DEVICE)
event_train_torch = torch.tensor(event_train, dtype=torch.int64, device=DEVICE)

mcmc = MCMC(
    NUTS(bayesian_competing_risks_model, target_accept_prob=MCMC_PARAMS['target_accept_prob'], jit_compile=False),
    num_samples=MCMC_PARAMS['num_samples'],
    warmup_steps=MCMC_PARAMS['num_warmup'],
    num_chains=MCMC_PARAMS['num_chains'],
)

mcmc.run(X=X_train_torch, durations=duration_train_torch, events=event_train_torch, n_features=X_train.shape[1])
print("MCMC completed!")

In [ ]:
# Get posterior samples
mcmc.summary()
posterior_samples = {k: v.cpu().numpy() for k, v in mcmc.get_samples().items()}
inference_data = az.from_pyro(mcmc)

print("\nPosterior shapes:")
for k, v in posterior_samples.items():
    print(f"  {k}: {v.shape}")

In [ ]:
# Trace plots
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for i, param in enumerate(['mu_D', 'sigma_D', 'mu_P', 'sigma_P']):
    samples = posterior_samples[param]
    axes[0, i].plot(samples, alpha=0.7)
    axes[0, i].set_title(f'Trace: {param}')
    axes[1, i].hist(samples, bins=50, density=True, alpha=0.7)
    axes[1, i].axvline(np.mean(samples), color='red', linestyle='--')
    axes[1, i].set_title(f'Posterior: {param}')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'bayesian_trace_baseline.png', dpi=150)
plt.show()

In [ ]:
# Coefficient summaries
baseline_df = pd.DataFrame([{
    'Parameter': p, 'Mean': np.mean(posterior_samples[p]), 'Median': np.median(posterior_samples[p]),
    'CI_2.5%': np.percentile(posterior_samples[p], 2.5), 'CI_97.5%': np.percentile(posterior_samples[p], 97.5)
} for p in ['mu_D', 'sigma_D', 'mu_P', 'sigma_P']])
print(baseline_df.to_string(index=False))

coef_summary_D = [{'Feature': f, 'Mean': np.mean(posterior_samples['theta_D'][:, i]),
    'CI_2.5%': np.percentile(posterior_samples['theta_D'][:, i], 2.5),
    'CI_97.5%': np.percentile(posterior_samples['theta_D'][:, i], 97.5),
    'Significant': not (np.percentile(posterior_samples['theta_D'][:, i], 2.5) < 0 < np.percentile(posterior_samples['theta_D'][:, i], 97.5))
} for i, f in enumerate(feature_cols)]

coef_summary_P = [{'Feature': f, 'Mean': np.mean(posterior_samples['theta_P'][:, i]),
    'CI_2.5%': np.percentile(posterior_samples['theta_P'][:, i], 2.5),
    'CI_97.5%': np.percentile(posterior_samples['theta_P'][:, i], 97.5),
    'Significant': not (np.percentile(posterior_samples['theta_P'][:, i], 2.5) < 0 < np.percentile(posterior_samples['theta_P'][:, i], 97.5))
} for i, f in enumerate(feature_cols)]

coef_df_D, coef_df_P = pd.DataFrame(coef_summary_D), pd.DataFrame(coef_summary_P)

---

## Model Wrapper & Predictions

In [ ]:
# Model wrapper for evaluation module
class BayesianModelWrapper:
    def __init__(self, posterior_samples, device='cpu'):
        self.posterior_samples_ = posterior_samples
        self.device = device
    
    def predict_cif(self, X, times, cause='default'):
        X = torch.tensor(X, dtype=torch.float32, device=self.device)
        times = torch.tensor(times, dtype=torch.float32, device=self.device)
        N, T = X.shape[0], len(times)
        
        ps = {k: torch.tensor(v, device=self.device) for k, v in self.posterior_samples_.items()}
        n_samples = len(ps['mu_D'])
        cif_samples = np.zeros((n_samples, N, T))
        
        for s in range(n_samples):
            eta_D = torch.matmul(X, ps['theta_D'][s])
            eta_P = torch.matmul(X, ps['theta_P'][s])
            for t_idx, t in enumerate(times):
                H_D = lognormal_cumulative_hazard(t, ps['mu_D'][s], ps['sigma_D'][s]) * torch.exp(eta_D)
                H_P = lognormal_cumulative_hazard(t, ps['mu_P'][s], ps['sigma_P'][s]) * torch.exp(eta_P)
                S_t = torch.exp(-H_D - H_P)
                cif = (H_D if cause == 'default' else H_P) / (H_D + H_P + 1e-10) * (1 - S_t)
                cif_samples[s, :, t_idx] = cif.cpu().numpy()
        
        return np.mean(cif_samples, 0), np.percentile(cif_samples, 2.5, 0), np.percentile(cif_samples, 97.5, 0)
    
    def predict_survival(self, X, times):
        X = torch.tensor(X, dtype=torch.float32, device=self.device)
        times = torch.tensor(times, dtype=torch.float32, device=self.device)
        N, T = X.shape[0], len(times)
        
        ps = {k: torch.tensor(v, device=self.device) for k, v in self.posterior_samples_.items()}
        n_samples = len(ps['mu_D'])
        surv_samples = np.zeros((n_samples, N, T))
        
        for s in range(n_samples):
            eta_D = torch.matmul(X, ps['theta_D'][s])
            eta_P = torch.matmul(X, ps['theta_P'][s])
            for t_idx, t in enumerate(times):
                H_D = lognormal_cumulative_hazard(t, ps['mu_D'][s], ps['sigma_D'][s]) * torch.exp(eta_D)
                H_P = lognormal_cumulative_hazard(t, ps['mu_P'][s], ps['sigma_P'][s]) * torch.exp(eta_P)
                surv_samples[s, :, t_idx] = torch.exp(-H_D - H_P).cpu().numpy()
        
        return np.mean(surv_samples, 0), np.percentile(surv_samples, 2.5, 0), np.percentile(surv_samples, 97.5, 0)

model = BayesianModelWrapper(posterior_samples, device='cpu')
print("Model wrapper created.")

In [ ]:
# Compute predictions
time_points = np.linspace(1, 180, 100)
cif_default_mean, cif_default_lower, cif_default_upper = model.predict_cif(X_test, time_points, 'default')
cif_prepay_mean, cif_prepay_lower, cif_prepay_upper = model.predict_cif(X_test, time_points, 'prepay')
surv_mean, surv_lower, surv_upper = model.predict_survival(X_test, time_points)
print(f"Predictions computed: CIF shape {cif_default_mean.shape}")

In [ ]:
# Plot sample predictions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
sample_idx = np.random.choice(len(X_test), 5, replace=False)

for idx in sample_idx:
    axes[0].plot(time_points, cif_prepay_mean[idx], alpha=0.7)
    axes[1].plot(time_points, cif_default_mean[idx], alpha=0.7)
    axes[2].plot(time_points, surv_mean[idx], alpha=0.7)

axes[0].set_title('Prepayment CIF'); axes[1].set_title('Default CIF'); axes[2].set_title('Survival')
for ax in axes: ax.set_xlabel('Time (months)'); ax.grid(True, alpha=0.3); ax.set_ylim(0, 1)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'bayesian_survival_curves.png', dpi=150)
plt.show()

---

## Performance Evaluation (using bayesian_evaluation module)

In [ ]:
# Comprehensive evaluation using the module
print("Running comprehensive evaluation...\n")

eval_results = evaluate_bayesian_model(
    model=model,
    X_train=X_train, durations_train=duration_train, events_train=event_train,
    X_test=X_test, durations_test=duration_test, events_test=event_test,
    time_horizons=TIME_HORIZONS,
    causes=['default', 'prepay']
)

print(format_evaluation_results(eval_results))

In [ ]:
# Plot C-index
fig, ax = plt.subplots(figsize=(10, 6))
horizons = TIME_HORIZONS
x = np.arange(len(horizons))
width = 0.35

prepay_vals = [eval_results['cindex']['prepay'].get(h, np.nan) for h in horizons]
default_vals = [eval_results['cindex']['default'].get(h, np.nan) for h in horizons]

ax.bar(x - width/2, prepay_vals, width, label='Prepayment', color='steelblue')
ax.bar(x + width/2, default_vals, width, label='Default', color='indianred')
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5)
ax.set_xticks(x); ax.set_xticklabels([f'τ={h}' for h in horizons])
ax.set_ylabel('C-index (IPCW)'); ax.set_ylim(0.4, 1.0); ax.legend()
ax.set_title('Bayesian PHM: Time-Dependent C-index')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'bayesian_time_dependent_cindex.png', dpi=150)
plt.show()

In [ ]:
# Calibration plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
for row, cause in enumerate(['prepay', 'default']):
    for col, tau in enumerate(TIME_HORIZONS):
        ax = axes[row, col]
        cal = eval_results['calibration'][cause].get(tau, {})
        if 'predicted_mean' in cal:
            pred, obs = cal['predicted_mean'], cal['observed_rate']
            valid = ~np.isnan(obs)
            if valid.sum() > 0:
                ax.scatter(pred[valid], obs[valid], alpha=0.7, s=100)
                ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
        ax.set_title(f'{cause.capitalize()} τ={tau}')
        ax.set_xlabel('Predicted'); ax.set_ylabel('Observed')
plt.suptitle('Calibration Plots', fontsize=14)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'bayesian_calibration.png', dpi=150)
plt.show()

---

## Save Results

In [ ]:
# Save artifacts
np.savez(MODELS_DIR / 'bayesian_phm_posterior.npz', **posterior_samples)
inference_data.to_netcdf(MODELS_DIR / 'bayesian_phm_inference.nc')
coef_df_D.to_csv(MODELS_DIR / 'bayesian_phm_coef_default.csv', index=False)
coef_df_P.to_csv(MODELS_DIR / 'bayesian_phm_coef_prepay.csv', index=False)
baseline_df.to_csv(MODELS_DIR / 'bayesian_phm_baseline.csv', index=False)
pd.DataFrame(eval_results['cindex']).to_csv(MODELS_DIR / 'bayesian_phm_cindex.csv')
with open(MODELS_DIR / 'bayesian_phm_scaler.pkl', 'wb') as f: pickle.dump(scaler, f)
with open(MODELS_DIR / 'bayesian_phm_features.pkl', 'wb') as f: pickle.dump(feature_cols, f)
print(f"Results saved to {MODELS_DIR}")

---

## Summary

In [ ]:
print("=" * 70)
print("BAYESIAN COMPETING RISKS PHM - SUMMARY")
print("=" * 70)
print(f"\nInference: MCMC (NUTS via Pyro/PyTorch)")
print(f"Data: Train {len(train_df):,}, Test {len(test_df):,}")
print(f"Features: {len(feature_cols)}")
print(f"\nC-index (IPCW):")
for cause in ['prepay', 'default']:
    print(f"  {cause.upper()}:")
    for tau, c in eval_results['cindex'][cause].items():
        print(f"    τ={tau}: {c:.4f}")
print(f"\nSignificant covariates:")
print(f"  Default: {', '.join([c['Feature'] for c in coef_summary_D if c['Significant']]) or 'None'}")
print(f"  Prepay: {', '.join([c['Feature'] for c in coef_summary_P if c['Significant']]) or 'None'}")